In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np


housing = fetch_california_housing()
X_train_full, X_test, y_train, y_test = train_test_split(housing.data, housing.target, random_state=42)
print(X_train_full.shape, X_test.shape)



2023-08-07 03:20:44.868844: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-07 03:20:45.340373: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-07 03:20:46.928585: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(15480, 8) (5160, 8)


In [2]:
class DeepAndWide(tf.keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)

        self.norm_layer_wide = tf.keras.layers.Normalization()
        self.norm_layer_deep =  tf.keras.layers.Normalization()
        self.hidden1 = tf.keras.layers.Dense(units, activation)
        self.hidden2 = tf.keras.layers.Dense(units, activation)
        self.main_output = tf.keras.layers.Dense(1)
        self.aux_output = tf.keras.layers.Dense(1)

    def call(self, inputs):
        input_wide, input_deep = inputs
        norm_wide = self.norm_layer_wide(input_wide)
        norm_deep = self.norm_layer_deep(input_deep)
        hidden1 = self.hidden1(norm_deep)
        hidden2 = self.hidden2(hidden1)
        concat = tf.keras.layers.concatenate([norm_wide, hidden2])
        output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)

        return output, aux_output
    
model = DeepAndWide(30, "relu", name="my_model")

model.norm_layer_wide.adapt(X_train_full)
model.norm_layer_deep.adapt(X_train_full)

optimiser = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=("mse", "mse"), loss_weights=(0.9, 0.1) ,optimizer=optimiser, metrics=["RootMeanSquaredError"])

2023-08-07 03:35:48.354840: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-07 03:35:48.356052: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
model.fit((X_train_full,X_train_full), (y_train, y_train) ,epochs=30 )

Epoch 1/30
484/484 [==============================] - 2s 2ms/step - loss: 1.2614 - output_1_loss: 1.2394 - output_2_loss: 1.4587 - output_1_root_mean_squared_error: 1.1133 - output_2_root_mean_squared_error: 1.2078
Epoch 2/30
484/484 [==============================] - 1s 2ms/step - loss: 0.4684 - output_1_loss: 0.4522 - output_2_loss: 0.6146 - output_1_root_mean_squared_error: 0.6724 - output_2_root_mean_squared_error: 0.7839
Epoch 3/30
484/484 [==============================] - 1s 2ms/step - loss: 0.4030 - output_1_loss: 0.3989 - output_2_loss: 0.4401 - output_1_root_mean_squared_error: 0.6315 - output_2_root_mean_squared_error: 0.6634
Epoch 4/30
484/484 [==============================] - 1s 2ms/step - loss: 0.3848 - output_1_loss: 0.3828 - output_2_loss: 0.4035 - output_1_root_mean_squared_error: 0.6187 - output_2_root_mean_squared_error: 0.6352
Epoch 5/30
484/484 [==============================] - 1s 2ms/step - loss: 0.3655 - output_1_loss: 0.3643 - output_2_loss: 0.3756 - output_1_

In [6]:
eval_results = model.evaluate((X_test,X_test), (y_test, y_test))

162/162 [==============================] - 0s 2ms/step - loss: 0.2982 - output_1_loss: 0.2981 - output_2_loss: 0.2992 - output_1_root_mean_squared_error: 0.5460 - output_2_root_mean_squared_error: 0.5470


In [8]:
print("loss main_output: ", eval_results[0])
print("RMSE main_output: ", eval_results[1])
print("loss aux_output: ", eval_results[2])
print("RMSE aux_output: ", eval_results[3])

loss main_output:  0.2982213795185089
RMSE main_output:  0.29811304807662964
loss aux_output:  0.2991962730884552
RMSE aux_output:  0.5459973216056824
